# Use Case 10: Reactome Pathways

Enrichment analysis is often used to narrow a large set of genes and focus on those in the same pathway. Further examination of the pathway can be aided by visualization. In this use case we will demonstrate how to visualize results on Reactome Pathways.

## Step 1: Importing packages
A variety of python packages must be imported in order to download data from cptac, run statistical analysis to find meaningful clusters of proteins, and then overlay pathway diagrams found on Reactome. cptac.utils is a hosts a great source of functions for completing a variety of complicated tasks. See `help(cptac.utils)`

In [1]:
import cptac
import pandas as pd
import cptac.utils as utils
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels as statsmodels
from IPython.display import Image

## Step 2: Getting the data

Even though the cptac package has been imported, we still need to load datasets. In the cell below we get the Renal Cancer proteomics data.

In [3]:
ds = cptac.Ccrcc()
df = ds.get_proteomics('umich')
df.head()

Name                     ARF5              M6PR             ESRRA  \
Database_ID ENSP00000000233.5 ENSP00000000412.3 ENSP00000000442.6   
Patient_ID                                                          
C3L-00004            0.248796          0.308289         -0.315825   
C3L-00010            0.245087          0.131992         -0.016811   
C3L-00011            0.576821          0.443678         -0.544452   
C3L-00026            0.293042          0.250350          0.142924   
C3L-00079            0.174315          0.026805         -0.663191   

Name                    FKBP4           NDUFAF7             FUCA2  \
Database_ID ENSP00000001008.4 ENSP00000002125.4 ENSP00000002165.5   
Patient_ID                                                          
C3L-00004            0.051528         -0.323475         -0.720420   
C3L-00010           -0.114728         -0.816992          0.104260   
C3L-00011            0.120776         -0.075035         -0.738826   
C3L-00026           -0.173812          0.589458          0.238597   
C3L-00079            0.075943         -0.240099         -0.293380   

Name                   DBNDD1              CFTR           CYP51A1  \
Database_ID ENSP00000002501.6 ENSP00000003084.6 ENSP00000003100.8   
Patient_ID                                                          
C3L-00004                 NaN         -0.832085         -0.629139   
C3L-00010                 NaN          1.266034         -0.573329   
C3L-00011                 NaN         -0.484732         -0.725274   
C3L-00026            0.957386         -3.178976          0.372489   
C3L-00079                 NaN          0.008966         -0.469829   

Name                    USP28  ...              ANK2                    \
Database_ID ENSP00000003302.4  ... ENSP00000500245.1 ENSP00000500275.1   
Patient_ID                     ...                                       
C3L-00004           -0.317251  ...          0.359093          1.609902   
C3L-00010           -0.172712  ...         -1.592004         -0.542294   
C3L-00011            1.079570  ...         -0.523301         -0.389680   
C3L-00026            0.404721  ...         -0.249875         -1.492455   
C3L-00079                 NaN  ...         -1.423105         -0.921139   

Name                    ATXN2             ETNK1              MYO6  \
Database_ID ENSP00000500313.1 ENSP00000500633.1 ENSP00000500710.1   
Patient_ID                                                          
C3L-00004            0.054346         -0.471825         -0.095824   
C3L-00010                 NaN         -0.208478         -0.143536   
C3L-00011                 NaN          0.550085         -1.617542   
C3L-00026                 NaN          0.005406         -0.281776   
C3L-00079                 NaN         -0.042932         -0.921862   

Name                     GBF1            CTNND1               PRX  \
Database_ID ENSP00000501064.1 ENSP00000501126.1 ENSP00000501261.1   
Patient_ID                                                          
C3L-00004            0.240579          0.689392               NaN   
C3L-00010            0.049889          0.499405          1.422138   
C3L-00011            0.485815         -0.327129               NaN   
C3L-00026            0.057764         -0.073197               NaN   
C3L-00079            0.385364               NaN               NaN   

Name                      WIZ           HSPA12A  
Database_ID ENSP00000501300.1 ENSP00000501491.1  
Patient_ID                                       
C3L-00004           -0.072279         -0.655139  
C3L-00010           -0.058976         -0.189293  
C3L-00011            0.344999         -0.688779  
C3L-00026           -0.159555         -0.023920  
C3L-00079            0.387207         -1.057636  

[5 rows x 11889 columns]



Using utils we can get data from a variety of large databases. Here the Hugo Gene Nomenclature database is queried for a list of proteins in the proteosome. 



In [4]:
hgnc_lists = utils.get_hgnc_protein_lists()
proteasome_proteins = sorted(set(hgnc_lists["Proteasome"]))
included_cols = (df.columns.get_level_values(0) & proteasome_proteins).drop_duplicates()
included_cols

C:\Users\sabme\AppData\Local\Temp\ipykernel_35088\519696161.py:3: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  included_cols = (df.columns.get_level_values(0) & proteasome_proteins).drop_duplicates()


Index(['PSMA4', 'PSMC4', 'PSME1', 'PSMD5', 'PSMD8', 'PSMA3', 'PSME2', 'PSMD10',
       'PSMD7', 'PSMF1', 'ADRM1', 'PSMB7', 'PSMC1', 'PSMA6', 'PSMD11', 'PSMB1',
       'PSMD3', 'PSMB6', 'PSMA5', 'PSMB4', 'PSMC2', 'PSME3', 'PSMD6', 'PSMC3',
       'PSMD1', 'PSMD2', 'PSMC5', 'PSMD12', 'PSMB10', 'PSMB5', 'PSMD4',
       'PSMA7', 'PSMB2', 'PSMB9', 'PSMB8', 'PSMA1', 'PSME4', 'PSMD14', 'PSMC6',
       'PSMD13', 'PSMD9', 'PSMB3'],
      dtype='object', name='Name')

## Step 3: Identify tumor/normal differences

Next we loop throught the list of protiens, seperating tumor and normal data from each other. A t-test is then performed on both lists of data. For each protein with a p-value of less than .05, the protein name, p-value and difference between tumor mena and normal mean is stored in seperate lists.

In [5]:
nameList =[]
pVal_list = []
deltaList = []
selected_proteins = df[included_cols]

for protein in selected_proteins.columns:
    data = selected_proteins[protein]
    normal = data[data.index.str.endswith('.N')]
    tumor = data[~data.index.str.endswith('.N')]
    pVal = stats.ttest_ind(normal, tumor, equal_var = False, nan_policy='omit').pvalue 
    
    if pVal < .05:
        name = protein[0]
        delta = tumor.mean() - normal.mean()
        nameList.append(name)
        pVal_list.append(pVal)
        deltaList.append(delta)
        nameList
     
print(nameList)

['PSMA4', 'PSMC4', 'PSME1', 'PSMD5', 'PSMD8', 'PSMA3', 'PSME2', 'PSMD10', 'PSMD7', 'PSMF1', 'PSMF1', 'ADRM1', 'PSMB7', 'PSMC1', 'PSMA6', 'PSMD11', 'PSMB1', 'PSMD3', 'PSMB6', 'PSMA5', 'PSMB4', 'PSMC2', 'PSME3', 'PSMD6', 'PSMC3', 'PSMD1', 'PSMD1', 'PSMD2', 'PSMC5', 'PSMD12', 'PSMB10', 'PSMB5', 'PSMD4', 'PSMD4', 'PSMA7', 'PSMB2', 'PSMB9', 'PSMB8', 'PSMA1', 'PSME4', 'PSMD14', 'PSMC6', 'PSMD13', 'PSMD9', 'PSMB3']


Benjamini & Hochberg/Yekutieli correction is applied to the list of p-values calculated in the previous cell. A dataframe is then constructed using the lists of names, p-values, and difference between tumor mean and normal mean. 

In [6]:
reject_null, adj_p, alpha_sidak, alpha_bonf = statsmodels.stats.multitest.multipletests(pVal_list, alpha=0.05, method='fdr_bh')
pVal_list = adj_p
zippedList =  list(zip(nameList, pVal_list, deltaList))
dfObj = pd.DataFrame(zippedList, columns =  ['Name', 'P-Value', 'Delta'])
dfObj = dfObj.set_index('Name')
print("Dataframe : " , dfObj, sep='\n')

Dataframe : 
             P-Value     Delta
Name                          
PSMA4   1.911611e-10  0.148198
PSMC4   1.391607e-09  0.112355
PSME1   2.008155e-13  0.316319
PSMD5   2.182605e-11  0.217871
PSMD8   7.052978e-09  0.138417
PSMA3   1.075243e-12  0.177574
PSME2   8.868141e-36  0.671231
PSMD10  1.370257e-56  0.468575
PSMD7   1.926944e-05  0.080727
PSMF1   2.042107e-05  0.184903
PSMF1   1.563592e-27  0.381929
ADRM1   4.086117e-03  0.068020
PSMB7   1.087052e-08 -0.263777
PSMC1   2.837536e-18  0.169574
PSMA6   4.235036e-24  0.277224
PSMD11  4.601922e-04  0.064925
PSMB1   1.976143e-29  0.278219
PSMD3   2.011238e-12  0.141735
PSMB6   5.243052e-27 -0.716784
PSMA5   2.114399e-35  0.302883
PSMB4   1.921437e-21  0.255068
PSMC2   4.943515e-10  0.109437
PSME3   8.587448e-06 -0.129204
PSMD6   9.883283e-13  0.130206
PSMC3   3.781102e-15  0.139832
PSMD1   1.100341e-05  0.082433
PSMD1   2.889130e-08  0.509608
PSMD2   1.523451e-03  0.060321
PSMC5   3.164878e-16  0.155371
PSMD12  1.703326e-37  0.33

Change in abundance (Delta column) is then simplified. Any Increase is assigned 1, decrease assigned -1, no change is set at 0.

In [7]:
dfObj.insert(1, "simplified_change", dfObj["Delta"])
dfObj["simplified_change"][dfObj["Delta"] > 0] = 1
dfObj["simplified_change"][dfObj["Delta"] < 0] = -1
dfObj["simplified_change"][dfObj["Delta"] == 0] = 0

Using the Benjamini & Hochberg/Yekutieli correction function we are able to also evaluate whether or not to reject the null. This is evaluated and added as a column in the dataframe.

In [8]:
dfObj = dfObj.assign(adj_p=adj_p, reject_null=reject_null)
sig_res = dfObj[dfObj["reject_null"]]
sig_genes = dfObj.index.get_level_values(0).tolist()
sig_res

,P-Value,simplified_change,Delta,adj_p,reject_null
Name,,,,,
PSMA4,1.911611e-10,1.0,0.148198,1.911611e-10,True
PSMC4,1.391607e-09,1.0,0.112355,1.391607e-09,True
PSME1,2.008155e-13,1.0,0.316319,2.008155e-13,True
PSMD5,2.182605e-11,1.0,0.217871,2.182605e-11,True
PSMD8,7.052978e-09,1.0,0.138417,7.052978e-09,True
PSMA3,1.075243e-12,1.0,0.177574,1.075243e-12,True
PSME2,8.868141e-36,1.0,0.671231,8.868141e-36,True
PSMD10,1.370257e-56,1.0,0.468575,1.370257e-56,True
PSMD7,1.926944e-05,1.0,0.080727,1.926944e-05,True


## Step 4: Display data with Reactome

Within utils from the cptac package we can use a function called "reactome_pathway_overlay" to query reactome and overlay the proteosome data we have found to be significant. A detailed guide on using this function can be read by calling `help(utils.reactome_pathway_overlay)`.
In this example I have chosen to overlay my data with the R-HSA-1236978 pathway ID. Determining desired pathway ID's in relation to your specific data can be done in a variety of methods such as gene enrichment analysis.
Finally, with the quality parameter set at 10 you can zoom in on areas of the pathway to see further detail.

In [9]:
expression_vals, image_path = utils.reactome_pathway_overlay(
    'R-HSA-1236978', 
    df=dfObj['simplified_change'], 
    analysis_token=None, 
    open_browser=False,
    export_path="test.png", 
    image_format='png', 
    display_col_idx=None,                      
    diagram_colors='Modern', 
    overlay_colors='Standard', 
    quality=10)
Image(image_path)